## LAB 3 tree-based classification 

### Adult-Income Dataset
[source] https://raw.githubusercontent.com/saravrajavelu/Adult-Income-Analysis/master/adult.csv :
Predict whether income exceeds $50K/yr based on census data

**Attribute information**
1.   age: continuous.
2.   workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.List item
3.  fnlwgt: continuous.
4.  education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
5.  education-num: continuous.
6.  marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
7.  occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
8. relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
9. race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
10. sex: Female, Male.
11. capital-gain: continuous.
12. capital-loss: continuous.
13. hours-per-week: continuous.
14. native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

**The outcome (Label)**
15. Income: >50K, <=50K

## อ่านชุดข้อมูล Adult Income

In [8]:
import pandas as pd
df = pd.read_csv("../Dataset/adult_income.csv")
df

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33958,37,Private,126675,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,57,United-States,<=50K
33959,46,Local-gov,175754,Masters,14,Divorced,Prof-specialty,Not-in-family,White,Female,0,1876,60,United-States,<=50K
33960,31,Private,121768,Assoc-acdm,12,Divorced,Adm-clerical,Unmarried,White,Female,0,0,35,Poland,<=50K
33961,23,Private,180052,HS-grad,9,Never-married,Other-service,Own-child,White,Male,0,0,40,United-States,<=50K


## ข้อ 1 จงวิเคราะห์ข้อมูลด้วย pandas เพื่อตอบคำถามข้อ 1.1 ถึง 1.6

In [9]:
# 1.1
samples = df['income'].value_counts()
samples

income
<=50K    25893
>50K      8069
Name: count, dtype: int64

In [10]:
# 1.2

categorical_features = df.select_dtypes(include=['object', 'category'])
categorical_features

,workclass,education,marital-status,occupation,relationship,race,gender,native-country,income
0,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,United-States,<=50K
1,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,United-States,<=50K
2,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,United-States,>50K
3,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,United-States,>50K
4,?,Some-college,Never-married,?,Own-child,White,Female,United-States,<=50K
...,...,...,...,...,...,...,...,...,...
33958,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,<=50K
33959,Local-gov,Masters,Divorced,Prof-specialty,Not-in-family,White,Female,United-States,<=50K
33960,Private,Assoc-acdm,Divorced,Adm-clerical,Unmarried,White,Female,Poland,<=50K
33961,Private,HS-grad,Never-married,Other-service,Own-child,White,Male,United-States,<=50K


In [11]:
# 1.3

age = df['age'].mean()
print("อายุของพนักงานเฉลี่ย = ",age)

อายุของพนักงานเฉลี่ย =  38.655566351617935


In [12]:
# 1.4

selected_columns = df.loc[
    (df['age'] > 40) &
    (df['marital-status'] == 'Never-married') &
    (df['income'] == '>50K'),
    ['age', 'marital-status', 'income']
]

print(selected_columns)

filtered_df = df[
    (df['age'] > 40) &
    (df['marital-status'] == 'Never-married') &
    (df['income'] == '>50K')
]

count = len(filtered_df)
print("\nมีจำนวนพนักงาน = ", count)


       age marital-status income
584     44  Never-married   >50K
665     43  Never-married   >50K
702     43  Never-married   >50K
788     45  Never-married   >50K
1448    43  Never-married   >50K
...    ...            ...    ...
32685   51  Never-married   >50K
32719   59  Never-married   >50K
33145   42  Never-married   >50K
33335   60  Never-married   >50K
33844   71  Never-married   >50K

[185 rows x 3 columns]

มีจำนวนพนักงาน =  185


In [13]:
# 1.5

Work_Hours_Per_Week = df['hours-per-week'].max()
print("จำนวนชั่วโมงทำงานต่อสัปดาห์มากที่สุด = ",Work_Hours_Per_Week)

จำนวนชั่วโมงทำงานต่อสัปดาห์มากที่สุด =  99


In [14]:
# 1.6

ms_value_questionMarks = (df == '?').sum()

featues_question_Marks = ms_value_questionMarks[ms_value_questionMarks > 0]

print("Features ที่มี missing values เป็น '?' :")
print(featues_question_Marks)

Features ที่มี missing values เป็น '?' :
workclass         1951
occupation        1957
native-country     595
dtype: int64


## ข้อ 2. เขียนฟังก์ชั่นจัดการ missing values ในชุดข้อมูลนี้

## ข้อ 3. แปลง categorical features ให้อยู่ในรูปของ binary features 

## ข้อ 4. แบ่งชุดข้อมูลดังกล่าวออกเป็น  80% และ 20% สำหรับฝึก (train) และทดสอบ (test) 

## 5. สร้างโมเดล Decision tree โดยใช้ชุดข้อมูล train กำหนดให้ใช้ entropy เป็น split criterion และความลึกสูงสุดของต้นไม้ไม่เกิน 2 ระดับ (level) 

## ข้อ 6. ความแม่นยำ (accuracy) ของโมเดล decision tree บนชุดข้อมูลทดสอบ 

### ข้อ 7. สร้างโมเดล random forest โดยกำหนดความลึกสูงสุดของต้นไม้เท่ากับ 3 ขณะที่จำนวนต้นไม้ย่อย (n_estimator) ใช้ GridSearchCV ค้นหาค่าที่ดีที่สุดบนชุดข้อมูล train กำหนดช่วงค่าอยู่ระหว่าง 6 ถึง 10

### 8.	แสดงต้นไม้ random forest ที่สร้างได้ 6 ต้นแรก   

## 9.	ความแม่นยำ (accuracy) ของโมเดล random forest บนชุดข้อมูลทดสอบ 

## ข้อ 10. ปรับปรุงโมเดล random forest จากข้อ 7 โดยใช้อัลกอริทึม XgBoost พร้อมแสดงค่าความแม่นยำบนชุดข้อมูลทดสอบ (test data) 

## ข้อ 11. ทดลองใช้ GridSearchCV เพื่อหาจำนวนต้นไม้ย่อยใน XgBoost ที่ให้ผลลัพธ์สูงที่สุด จากการกำหนดค่าความลึกตั้งแต่ 2 ถึง 4 ระดับ และจำนวนต้นไม้ย่อยตั้งแต่  1 ถึง 20 ต้น 

## ข้อ 12.	บันทึกโมเดล XGboost ในรูปแบบของไฟล์ pickle ตั้งชื่อ XgB.pkl.